In [6]:
import numpy as np
import torch
from models import *
from inception_score import get_inception_score
from torch.utils.data import TensorDataset, DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Resize, Normalize

In [2]:
G_models = ['mnist_dcgan/G_10.pth', 'mnist_bigan/G_10.pth', 'mnist_logan_b/G_10.pth']
E_models = ['mnist_dcgan/Epost_1.pth', 'mnist_bigan/Epost_1.pth', 'mnist_logan_b/E_10.pth']

In [3]:
num_images = 2000

In [4]:
dataset = MNIST(root='.', download=True, transform=Compose([Resize(32), ToTensor(), Normalize((0.5,),(0.5,))]))
dataloader = DataLoader(dataset, batch_size=num_images, shuffle=True)

In [5]:
for (G_file, E_file) in zip(G_models, E_models):
    G = torch.load(G_file).to('cuda:1').eval()
    E = torch.load(E_file).to('cuda:1').eval()
    
    real = next(iter(dataloader))[0]
    latents = E(real.to('cuda:1')).detach()
    recon = ((G(latents).cpu().detach()+1)*127).squeeze().numpy()
    images = np.zeros((num_images, 3, 32, 32))
    images[:,0,:,:] = recon.copy()
    images[:,1,:,:] = recon.copy()
    images[:,2,:,:] = recon.copy()
    images = images.astype(np.uint8)
    print(G_file, get_inception_score(images))

Calculating Inception Score with 2000 images in 10 splits
Inception Score calculation time: 71.542205 s
mnist_dcgan/G_10.pth (2.1292377, 0.0758566)
Calculating Inception Score with 2000 images in 10 splits
Inception Score calculation time: 71.409925 s
mnist_bigan/G_10.pth (1.0015687, 0.00012461824)
Calculating Inception Score with 2000 images in 10 splits
Inception Score calculation time: 71.204535 s
mnist_logan_b/G_10.pth (2.002102, 0.06732421)
